In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
!unzip "drive/My Drive/Proyecto IA II/kaggle_bee_vs_wasp.zip"

Se han truncado las últimas 5000 líneas del flujo de salida.
  inflating: kaggle_bee_vs_wasp/other_noinsect/576855.jpg  
  inflating: kaggle_bee_vs_wasp/other_noinsect/576993.jpg  
  inflating: kaggle_bee_vs_wasp/other_noinsect/577087.jpg  
  inflating: kaggle_bee_vs_wasp/other_noinsect/577126.jpg  
  inflating: kaggle_bee_vs_wasp/other_noinsect/577128.jpg  
  inflating: kaggle_bee_vs_wasp/other_noinsect/577283.jpg  
  inflating: kaggle_bee_vs_wasp/other_noinsect/577310.jpg  
  inflating: kaggle_bee_vs_wasp/other_noinsect/577643.jpg  
  inflating: kaggle_bee_vs_wasp/other_noinsect/577807.jpg  
  inflating: kaggle_bee_vs_wasp/other_noinsect/577847.jpg  
  inflating: kaggle_bee_vs_wasp/other_noinsect/578261.jpg  
  inflating: kaggle_bee_vs_wasp/other_noinsect/578295.jpg  
  inflating: kaggle_bee_vs_wasp/other_noinsect/578441.jpg  
  inflating: kaggle_bee_vs_wasp/other_noinsect/578580.jpg  
  inflating: kaggle_bee_vs_wasp/other_noinsect/578650.jpg  
  inflating: kaggle_bee_vs_wasp/other_n

In [ ]:
#Librerias
import os
import shutil
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from tensorflow.keras.metrics import categorical_accuracy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import BatchNormalization, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score, classification_report, roc_auc_score

In [ ]:
metadata = pd.read_csv("/content/kaggle_bee_vs_wasp/labels.csv")
metadata.head()

,id,path,is_bee,is_wasp,is_otherinsect,is_other,photo_quality,is_validation,is_final_validation,label
0,1,bee1\10007154554_026417cfd0_n.jpg,1,0,0,0,1,0,0,bee
1,2,bee1\10024864894_6dc54d4b34_n.jpg,1,0,0,0,1,0,1,bee
2,3,bee1\10092043833_7306dfd1f0_n.jpg,1,0,0,0,1,1,0,bee
3,4,bee1\1011948979_fc3637e779_w.jpg,1,0,0,0,1,0,1,bee
4,5,bee1\10128235063_dca17db76c_n.jpg,1,0,0,0,1,0,0,bee


In [ ]:
metadata = metadata.drop(metadata[metadata["label"]=="other"].index.values)
metadata.head()

,id,path,is_bee,is_wasp,is_otherinsect,is_other,photo_quality,is_validation,is_final_validation,label
0,1,bee1\10007154554_026417cfd0_n.jpg,1,0,0,0,1,0,0,bee
1,2,bee1\10024864894_6dc54d4b34_n.jpg,1,0,0,0,1,0,1,bee
2,3,bee1\10092043833_7306dfd1f0_n.jpg,1,0,0,0,1,1,0,bee
3,4,bee1\1011948979_fc3637e779_w.jpg,1,0,0,0,1,0,1,bee
4,5,bee1\10128235063_dca17db76c_n.jpg,1,0,0,0,1,0,0,bee


In [ ]:
train, val_test = train_test_split(metadata, test_size=0.2, stratify = metadata["label"])
validation, test = train_test_split(val_test, test_size=0.5, stratify = val_test["label"])

In [ ]:
metadata_labels = metadata["label"]
metadata_labels_bee = metadata_labels[metadata_labels=="bee"]
metadata_labels_insect = metadata_labels[metadata_labels=="insect"]
metadata_labels_wasp = metadata_labels[metadata_labels=="wasp"]

print("Total bees: ", str(metadata_labels_bee.count()))
print("Total insects: ", str(metadata_labels_insect.count()))
print("Total wasps: ", str(metadata_labels_wasp.count()))

Total bees:  3183
Total insects:  2439
Total wasps:  4943


In [ ]:
print("Train: " + str(train.shape[0]))
print("Validation: " + str(validation.shape[0]))
print("Test: " + str(test.shape[0]))
print("")
print("Train: " + str(train.shape[0] / metadata.shape[0]))
print("Validation: " + str(validation.shape[0] / metadata.shape[0]))
print("Test: " + str(test.shape[0] / metadata.shape[0]))

Train: 8452
Validation: 1056
Test: 1057

Train: 0.8
Validation: 0.09995267392333175
Test: 0.10004732607666825


In [ ]:
#Now make new directories for train/val/test images.

base_dir = "base_dir"
os.mkdir(base_dir)

train_dir = os.path.join(base_dir, "image_train")
os.mkdir(train_dir)

val_dir = os.path.join(base_dir, "image_val")
os.mkdir(val_dir)

test_dir = os.path.join(base_dir, "image_test")
os.mkdir(test_dir)

labels = list(metadata["label"].unique())

for label in labels:
    label_path_train = os.path.join(train_dir, label)
    os.mkdir(label_path_train)
    label_path_val = os.path.join(val_dir, label)
    os.mkdir(label_path_val)
    label_path_test = os.path.join(test_dir, label)
    os.mkdir(label_path_test)

In [ ]:
#Copy the images to the new directory.

image_dir = "/content/kaggle_bee_vs_wasp"

for i in train["id"].values:
    image_folder, image_name = train["path"][i-1].split("\\",1)
    src_dir = os.path.join(image_dir,image_folder,image_name)
    dst_dir = os.path.join(train_dir, train["label"][i-1], image_name)
    shutil.copyfile(src_dir, dst_dir)

for i in validation["id"].values:
    image_folder, image_name = validation["path"][i-1].split("\\",1)
    src_dir = os.path.join(image_dir,image_folder,image_name)
    dst_dir = os.path.join(val_dir, validation["label"][i-1], image_name)
    shutil.copyfile(src_dir, dst_dir)
    
for i in test["id"].values:
    image_folder, image_name = test["path"][i-1].split("\\",1)
    src_dir = os.path.join(image_dir,image_folder,image_name)
    dst_dir = os.path.join(test_dir, test["label"][i-1], image_name)
    shutil.copyfile(src_dir, dst_dir)

In [ ]:
train_bee = np.shape(os.listdir("/content/base_dir/image_train/bee"))[0]
train_insect = np.shape(os.listdir("/content/base_dir/image_train/insect"))[0]
train_wasp = np.shape(os.listdir("/content/base_dir/image_train/wasp"))[0]
train_total = train_bee + train_insect + train_wasp

validation_bee = np.shape(os.listdir("/content/base_dir/image_val/bee"))[0]
validation_insect = np.shape(os.listdir("/content/base_dir/image_val/insect"))[0]
validation_wasp = np.shape(os.listdir("/content/base_dir/image_val/wasp"))[0]
validation_total = validation_bee + validation_insect + validation_wasp

test_bee = np.shape(os.listdir("/content/base_dir/image_test/bee"))[0]
test_insect = np.shape(os.listdir("/content/base_dir/image_test/insect"))[0]
test_wasp = np.shape(os.listdir("/content/base_dir/image_test/wasp"))[0]
test_total = test_bee + test_insect + test_wasp

print("Total train bee images: " + str(train_bee))
print("Total train insect images: " + str(train_insect))
print("Total train wasp images: " + str(train_wasp))
print("")
print("Total validation bee images: " + str(validation_bee))
print("Total validation insect images: " + str(validation_insect))
print("Total validation wasp images: " + str(validation_wasp))
print("")
print("Total test bee images: " + str(test_bee))
print("Total test insect images: " + str(test_insect))
print("Total test wasp images: " + str(test_wasp))
print("")
print("Total train images: " + str(train_total))
print("Total validation images: " + str(validation_total))
print("Total test images: " + str(test_total))

Total train bee images: 2547
Total train insect images: 1951
Total train wasp images: 3954

Total validation bee images: 318
Total validation insect images: 244
Total validation wasp images: 494

Total test bee images: 318
Total test insect images: 244
Total test wasp images: 495

Total train images: 8452
Total validation images: 1056
Total test images: 1057


In [ ]:
#Comprimir dataset

!zip -r no_data_aug_3_classes.zip base_dir

Se han truncado las últimas 5000 líneas del flujo de salida.
  adding: base_dir/image_train/wasp/G00065.jpg (deflated 0%)
  adding: base_dir/image_train/wasp/4895591778_a5b9c4c858_n.jpg (deflated 0%)
  adding: base_dir/image_train/wasp/F00413.jpg (deflated 0%)
  adding: base_dir/image_train/wasp/G00145.jpg (deflated 0%)
  adding: base_dir/image_train/wasp/6066310112_918a2256e6_m.jpg (deflated 0%)
  adding: base_dir/image_train/wasp/G00320.jpg (deflated 0%)
  adding: base_dir/image_train/wasp/49882717288_49d827bff3_m.jpg (deflated 4%)
  adding: base_dir/image_train/wasp/I00346.jpg (deflated 10%)
  adding: base_dir/image_train/wasp/183052665_f26ae9dd6e_n.jpg (deflated 0%)
  adding: base_dir/image_train/wasp/J00124.jpg (deflated 0%)
  adding: base_dir/image_train/wasp/W00471.jpg (deflated 0%)
  adding: base_dir/image_train/wasp/I00459.jpg (deflated 16%)
  adding: base_dir/image_train/wasp/J00101.jpg (deflated 0%)
  adding: base_dir/image_train/wasp/33935270924_738a50e986_n.jpg (deflated 3

In [ ]:
from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload

In [ ]:
#Autenticar en el drive para copiar

auth.authenticate_user()

In [ ]:
drive_service = build('drive', 'v3')

def save_file_to_drive(name, path):
  file_metadata = {
      'name': name,
      'mimeType': 'application/octet-stream'
     }

  media = MediaFileUpload(path, 
                    mimetype='application/octet-stream',
                    resumable=True)

  created = drive_service.files().create(body=file_metadata,
                                   media_body=media,
                                   fields='id').execute()

  print('File ID: {}'.format(created.get('id')))

  return created

In [ ]:
#Save file in Drive

save_file_to_drive('no_data_aug_3_classes.zip', 'no_data_aug_3_classes.zip')

File ID: 1h4-4pAOWXN2Cdzokvr-c8OV2de9oqmir


{'id': '1h4-4pAOWXN2Cdzokvr-c8OV2de9oqmir'}